In [1]:
%cd /content/
!git clone https://github.com/So-AI-love/Persian-text-to-speech
%cd Persian-text-to-speech


/content
Cloning into 'Persian-text-to-speech'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 172 (delta 2), reused 0 (delta 0), pack-reused 166
Receiving objects: 100% (172/172), 3.53 MiB | 3.18 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/Persian-text-to-speech


In [6]:
# !rm -rf  /content/Persian-text-to-speech

# Youtube Audio Scraing for building the dataset:
Install youtube-dl python mudule and use it like this:



```
! youtube-dl --extract-audio --audio-format wav  https://www.youtube.com/watch?v=kenujZ7WkUM
```



In [2]:
! pip install youtube-dl 
# ! youtube-dl --extract-audio --audio-format wav  https://www.youtube.com/watch?v=kenujZ7WkUM

     |████████████████████████████████| 1.8MB 2.6MB/s 


In [ ]:
# !youtube-dl --download-archive archive.txt "https://www.youtube.com/user/YouTube/playlists?app=desktop"

#Scrap huge data form :

based of this post:
https://askubuntu.com/questions/1027209/youtube-dl-search-keyword-and-only-download-urls


In [3]:
# %cd /content
!mkdir Database
%cd Database

/content/Persian-text-to-speech/Database


In [ ]:
!youtube-dl "ytsearch10:ابی" --get-id  --write-info-json --write-annotation --write-thumbnail --write-sub --skip-download

In [6]:
! youtube-dl --extract-audio --audio-format wav -o '%(title)s.%(ext)s' --restrict-filenames  "ytsearch500:  خواننده ابی" 


[download] 100% of 11.24MiB in 00:25
[ffmpeg] Destination: _.wav



In [ ]:
%cd ..
!git add .
!git commit -am " wav Database added"
!git push -u origin master

/content/Persian-text-to-speech


#Alternative Scraping metode by Selenium:
```
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)
wd.get("https://www.website.com")
print(wd.page_source)  # re
```

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)
wd.get("https://www.website.com")
print(wd.page_source)  # re

In [ ]:
import sys
from tqdm import tqdm
import tensorflow as tf
sys.path.append('tools')
%load_ext autoreload
%autoreload 2
from model_graph_test import model

In [ ]:
#use training_text2sp for training text to mel
#use training_superresolution for training super resolution network
gr=model('...DATA PATH...','training_text2sp')

In [ ]:
#The supervisor takes care of session initialization and all that jazz     
# Use tqdm for progress bar
#https://github.com/petewarden/tensorflow_makefile/blob/master/tensorflow/g3doc/api_docs/python/functions_and_classes/tf.train.Supervisor.md
logdir = 'logs/text-to-spec'
#config = tf.ConfigProto(allow_soft_placement = True)
sv = tf.train.Supervisor(logdir=logdir, save_model_secs=0, global_step=gr.global_step)
with sv.managed_session() as sess:
    while True:
            for _ in tqdm(range(gr.num_batch), total=gr.num_batch, ncols=70, leave=False, unit='b'):
                global_s,_=sess.run([gr.global_step,gr.train_operation])
                #print(global_s)
                if global_s % 1000 == 0:
                    sv.saver.save(sess, logdir + '/model_gs_{}'.format(str(global_s // 1000).zfill(3) + "k"))